## Homework 6
---

- Calculate the Movie Similarities on the ml-10M dataset to find and show the Top 10 most similar movies to Toy Story.
- The data must be read in from Amazon S3.
- Use PySpark DataFrames or RDDs
- Use cosine similarity

In [1]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1740798541661_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1740798541661_0002/container_1740798541661_0002_01_000001/tmp/spark-88eb24a8-0cae-482a-a7ad-9387b93f108d
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.18 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import struct, col
import numpy as np
import math

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Initialize a Spark session
spark = SparkSession.builder.appName("MovieSimilarity").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Define the paths
ratings_path = "s3://csc555-jaewon/ml-10M100K/ratings.dat"
movies_path = "s3://csc555-jaewon/ml-10M100K/movies.dat"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Read ratings and movies data into DataFrames
ratings_df = spark.read.option("delimiter", "::").csv(ratings_path, inferSchema = True)
movies_df = spark.read.option("delimiter", "::").csv(movies_path, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Rename columns
ratings_df = ratings_df.toDF("userId", "movieId", "rating", "timestamp")
movies_df = movies_df.toDF("movieId", "title", "genres")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ratings_df.show(5)
movies_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    122|   5.0|838985046|
|     1|    185|   5.0|838983525|
|     1|    231|   5.0|838983392|
|     1|    292|   5.0|838983421|
|     1|    316|   5.0|838983392|
+------+-------+------+---------+
only showing top 5 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
# Group by movieId and aggregate the userId and rating into a list of structs
user_movie_matrix = (ratings_df
                     .groupBy("movieId") # Group by the movieId
                     .agg(F.collect_list(struct(col("userId").cast("integer").alias("userId"), # Cast userId to integer
                                                col("rating").cast("double").alias("rating"))) # Cast rating to double
                           .alias("ratings"))) # Alias the collected list as 'ratings'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Function to convert the ratings into a user-item vector
def ratings_to_vector(ratings, n_users):

    # Sort the ratings by userId
    pairs = [(r['userId'] - 1, float(r['rating'])) for r in ratings]
    pairs.sort(key = lambda x: x[0])

    # Extract user indices and corresponding ratings
    indices = [p[0] for p in pairs]
    values = [p[1] for p in pairs]

    # Initialize a zeroed vector with length n_users
    vector = np.zeros(n_users)

    # Populate the vector with ratings based on the sorted indices
    for idx, value in zip(indices, values):
        vector[idx] = value

    return vector.tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Get the maximum user ID to determine the size of the vector
max_user_id = ratings_df.agg(F.max("userId")).collect()[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Define a UDF to apply the ratings_to_vector function
ratings_to_vector_udf = F.udf(lambda x: ratings_to_vector(x, max_user_id), ArrayType(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Apply the UDF to the user_movie_matrix DataFrame
movie_vectors = user_movie_matrix.select("movieId", ratings_to_vector_udf("ratings").alias("features"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Extract the feature vector for the movie "Toy Story"
toy_story_vector = movie_vectors.filter(col("movieId") == 1).select("features").collect()[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Function to compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    
    # Check if the lengths of the two vectors are the same
    if len(vec1) != len(vec2):
        return 0.0

    # Find the number of common non-zero elements between the vectors
    common_users = sum([1 for v1, v2 in zip(vec1, vec2) if v1 != 0 and v2 != 0])

    # If there are fewer than 10 common users, return similarity of 0
    if common_users < 10:
        return 0.0

    # Calculate the dot product of the two vectors
    dot_product = sum([v1 * v2 for v1, v2 in zip(vec1, vec2)])

    # Calculate the norms of the individual vectors
    norm1 = math.sqrt(sum([v1 ** 2 for v1 in vec1]))
    norm2 = math.sqrt(sum([v2 ** 2 for v2 in vec2]))

    return dot_product / (norm1 * norm2) if norm1 * norm2 != 0 else 0.0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Define a UDF for cosine similarity to apply on the movie vectors
cosine_similarity_udf = F.udf(lambda x: cosine_similarity(toy_story_vector, x), DoubleType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Calculate the similarity between "Toy Story" and other movies
similarities = movie_vectors.filter(col("movieId") != 1).select("movieId", cosine_similarity_udf("features").alias("score"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Join the movie similarity results with the movie titles and sort by similarity score
result = (similarities.join(movies_df, "movieId")
          .select(F.lit("Toy Story").alias("Movie Name"), # Set "Toy Story" as the reference movie
                  col("title").alias("Similar Movies"),
                  col("score").alias("score"))
          .orderBy(F.desc("score")) # Sort by descending similarity score
          .limit(10)) # Limit the results to the top 10 most similar movies

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Display the top 10 similar movies
result.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------------------------------------------------+------------------+
|Movie Name|Similar Movies                                              |score             |
+----------+------------------------------------------------------------+------------------+
|Toy Story |Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977)|0.5803460720322966|
|Toy Story |Independence Day (a.k.a. ID4) (1996)                        |0.5599535411310438|
|Toy Story |Star Wars: Episode VI - Return of the Jedi (1983)           |0.5451452000679463|
|Toy Story |Mission: Impossible (1996)                                  |0.5400152265083048|
|Toy Story |Willy Wonka & the Chocolate Factory (1971)                  |0.5283965606243346|
|Toy Story |Back to the Future (1985)                                   |0.5243306398074609|
|Toy Story |Aladdin (1992)                                              |0.5213231906632042|
|Toy Story |12 Monkeys (Twelve Monkeys) (1995)                        